In [1]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phpipamadmin",
  database="flowdata"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('netflow',)
('netflows',)
('no_part_tab',)


In [3]:
mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

mycursor.close()

('netflow',)
('netflows',)


True

In [3]:
for x in mycursor:
    print(x)

In [5]:
mycursor = mydb.cursor()

mycursor.execute("drop table no_part_tab;")

mycursor.close()

True

In [6]:
mycursor = mydb.cursor()
# Non-partitioned table
mycursor.execute("CREATE TABLE no_part_tab(c1 int default NULL, c2 varchar(30) default NULL, c3 date not null)default charset utf8;")

# Partitioned table
mycursor.execute("CREATE TABLE part_tab \
(c1 int default NULL, c2 varchar(30) default NULL, c3 date not null)\
default charset utf8 \
PARTITION BY RANGE(year(c3))\
(PARTITION p0 VALUES LESS THAN (1995),\
PARTITION p1 VALUES LESS THAN (1996),\
PARTITION p2 VALUES LESS THAN (1997) ,\
PARTITION p3 VALUES LESS THAN (1998) ,\
PARTITION p4 VALUES LESS THAN (1999) ,\
PARTITION p5 VALUES LESS THAN (2000) ,\
PARTITION p6 VALUES LESS THAN (2001) ,\
PARTITION p7 VALUES LESS THAN (2002) ,\
PARTITION p8 VALUES LESS THAN (2003) ,\
PARTITION p9 VALUES LESS THAN (2004) ,\
PARTITION p10 VALUES LESS THAN (2010),\
PARTITION p11 VALUES LESS THAN (MAXVALUE) );")

mycursor.close()

True

In [12]:
mycursor = mydb.cursor()
mycursor.execute("DELIMITER &&  ")

mycursor.execute("CREATE PROCEDURE load_part_tab() \
  begin \
  declare v int default 0; \
  while v < 800000 \
  do \
  insert into part_tab \
  value (v, 'testingpartitions', adddate('1995-01-01',(rand(v)*36520)mod 3652)); \
  set v = v + 1; \
  end while; \
end && ")
mycursor.execute("DELIMITER ; ")
mycursor.close()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'DELIMITER &&' at line 1

In [13]:
mycursor = mydb.cursor()
mycursor.execute("call load_part_tab(); ")
mycursor.close()

True

In [14]:
mycursor = mydb.cursor()
mycursor.execute("insert into no_part_tab select * from part_tab; ")
mycursor.close()

True